In [7]:
import os
from Bio import SeqIO, Entrez
import pandas as pd
import csv
import argparse
import copy
import re

In [115]:

'''
Downloads entries from GenBank Nucleotide by given query
Saves entries to file in GenBank format
Input:
    query - str - query for search in GenBank
Output:
    outfile - str - shortname of output file with sequences in genbank format
'''
def fetch_seq_from_Nucleotide(query, outfile):

    Entrez.email = "A.N.Other@example.com"
    # list with ids obtained by query
    record = Entrez.read(Entrez.esearch(db="nucleotide", term=query, idtype="acc", RetMax=1000000))
    id_list = record['IdList']
    print("Query to GenBank Nucleotide database:\"{}\"".format(query))
    print("Number of records found: {}".format(record["Count"]))
    print("{} records will be downloaded".format(len(id_list)))
    # output file with sequences will be saved in current working dir
    f_name = os.path.abspath(outfile)

    # output file with sequences in fasta format
    file_out = open(f_name, 'w')
    
    # Saving sequences 
    
    i = 0
    # fetch each sequence from GenBank by its ID and writes to ouput file
    for id in id_list:
        i+=1
        if i % 500 ==0:
            print("Downloaded {} sequences".format(i))
        handle = Entrez.efetch(db="nucleotide", id=id, rettype="genbank", retmode="text")
        for line in handle:
            file_out.write(line)
        handle.close()
    file_out.close()
    print('Finished')
    return(f_name)

In [116]:
def fetch_metadata_from_gb(input_file):
    '''
    For each entry in GenBank file retrieves the following data:

    # Identifiers
    Accession
    GenBank title (DEFINITION)

    # Classification
    Organism name
    Species
    Isolate
    Strain
    Family
    Lineage

    #Sequence quality
    Length
    Strand

    Num of annotated CDS

    ORF1a
    ORF1b
    ORF2
    coords
    
    #Sources
    Country
    Tissue/Specimen/Source
    Collection date
    Release date
    Submitters
    
    '''

    '''
     dictionary
     entries_data[GenBank Accession] = {}
     entries_data[GenBank Accession].keys() = [
                                                'Version',
                                                'GenBank title'
                                                'Organism name',
                                                'Species',
                                                'Family',
                                                'Virus Lineage',
                                                'Length',
                                                'Isolate',
                                                'Strain',
                                                'Strand',
                                                'Geo location'
                                                'Country',
                                                'Tissue/Specimen/Source',
                                                'Host',
                                                'Collection date',
                                                'Release date',
                                                'Submitters',
                                                'Sequencing Technology'
                                                'CDS number',
                                                'Artificial',
                                                'UNVERIFIED'
                                                ]
    '''
    entries_data = {}

    source_qualifiers = ["strain", "isolate", "geo_loc_name", "country", "collection_date", "isolation_source", "host"]
    source_qualifiers_columns = ["Strain", "Isolate", "Geo location", "Country", "Collection date", "Tissue/Specimen/Source", "Host"]

    entries = SeqIO.parse(os.path.abspath(input_file), "genbank")

    for entry in entries:
        entries_data[entry.name] = {}
        entries_data[entry.name]['Version'] = entry.id
        entries_data[entry.name]['GenBank title'] = entry.description

        entries_data[entry.name]['Release date'] = entry.annotations['date']
        entries_data[entry.name]['Organism name'] = entry.annotations['organism']
        
        entries_data[entry.name]['Virus Lineage'] = ';'.join(entry.annotations['taxonomy'])
        num_taxa = len(entry.annotations['taxonomy'])
        if num_taxa == 9:
            entries_data[entry.name]['Species'] = entry.annotations['taxonomy'][-1]
        else:
             entries_data[entry.name]['Species'] = 'NA'
        entries_data[entry.name]['Family'] = entry.annotations['taxonomy'][6]
        
        entries_data[entry.name]['Length'] = len(entry)

        for reference in entry.annotations['references']:
            if reference.title == 'Direct Submission':
                entries_data[entry.name]['Submitters'] = reference.authors

        count_cds = 0
        for feature in entry.features:
            if feature.type == 'source':
                for qualif, colname in zip(source_qualifiers, source_qualifiers_columns):
                    if qualif in feature.qualifiers:
                        entries_data[entry.name][colname] = feature.qualifiers[qualif][0]
                    else:
                        entries_data[entry.name][colname] = 'NA'
                if 'environmental_sample' in feature.qualifiers:
                    entries_data[entry.name]['Environmental'] = 'Yes'
                else:
                    entries_data[entry.name]['Environmental'] = 'No'
                    
            if feature.type == 'CDS':
                
                count_cds +=1
                
        entries_data[entry.name]['CDS count'] = count_cds


        artificial_condition1 = ("patent" in entry.description.lower()) or ("oligonucleotide" in entry.description.lower())
        artificial_condition2 = "crispr" in entry.description.lower()
        artificial_condition3 = "method" in entry.description.lower()
        artificial_condition4 = ("modified microbial nucleic acid" in entry.description.lower()) or ("nonfunctional" in entry.description.lower())

        
        
        if artificial_condition1 or artificial_condition2 or artificial_condition3 or artificial_condition4:
            entries_data[entry.name]['Artificial'] = "Yes"
        else:
            entries_data[entry.name]['Artificial'] = "No"


        if 'UNVERIFIED' in entry.description:
            entries_data[entry.name]['UNVERIFIED'] = "Yes"
        else:
            entries_data[entry.name]['UNVERIFIED'] = "No"

    meta_dataframe = pd.DataFrame.from_dict(entries_data, orient='index')
    
    meta_dataframe.to_csv(os.path.splitext(os.path.abspath(input_file))[0] + '.csv')
    #country_map - file with abbreviations of countries
    return meta_dataframe


In [117]:
#custom csv parser 
def read_csv(file_name, strip_it=True):
    if not os.path.exists(file_name):
        return {}

    def strip(value):
        return strip_it and value.strip() or value

    with open(file_name) as csvfile:
        reader = csv.DictReader(csvfile,
                                delimiter=",", 
                                fieldnames=["base", "new"])
        result = {}
        for row in reader:

            result[strip(row["base"])] = strip(row["new"])
        csvfile.close()
        return result

def map_feature(feature, feature_map):
    '''
    feature_map - dictionary, e.g. feature_map['Italia']='ITA'
    feature - possible key from feature_map
    return value if key is in feature_map
    '''
    for k, v in feature_map.items():
        if feature.lower() == k.lower():
            return v
    return feature


def map_keywords(stri, keywords_dict):
    '''
    stri - GenBank DEFINITION, string
    keywords_dict - dictionary with keywords that can indicate which ORF is covered  by the sequence
    return list of unique ORF codes if the corresponding keywords werencountered in DEFINITION
    '''
    codes = []
    for word, code in keywords_dict.items():
        if word.lower() in stri.lower():
            codes.append(code)
    codes = list(set(codes))
    return codes

    
def orf_coord(input_file, orf_map):
    '''
    Retrieves the coordinates of ORFs
    
    Input:
        input_file - file with nucleotide sequences in genbank-format
        orf_map - csv file annotation of orfs and their codes
    Output:
        coord_file - file with coordinates
    '''

    # dictionary with possiple names of astrovirus ORFs indicated in note, gene and product qualifiers
    # orf_dict["ORF1a"] = "1A"
    orf_dict = read_csv(orf_map)

    # ORFs which coordinates this script will extract
    orf_types_final = ['1A', '1B', '2']
    # ORFs that can be met
    orf_types = ['1A', '1B', '1AB', '2']

    # name of output file with ORF coordinates
    out_file_name_temp = os.path.splitext(os.path.abspath(input_file))[0]
    out_file_name = out_file_name_temp  + '_orf-coords.csv'
    print(out_file_name)

    # Keywords in description that can indicate which ORF is covered  by the sequence
    description_keywords = { "ORF2-like": "2", 
                            "ORF2": "2",
                            "precapsid":"2", 
                            "CP sequence": "2", 
                            "capsid": "2",
                            "ORF1a-like": "1A", 
                            "nsp1a-like": "1A", 
                            "ORF1A": "1A",
                            "1b-like": "1B", 
                            "RNA polymerase": "1B", 
                            "RdRp": "1B",
                            "ORF1b": "1B"
                            }
    
    # ENTRIES with known errors in annotation
    exceptions_ORF2 = ["MN725025", "KY047739", "S68561", "KP404151","KP404152"] # the last CDS in conflict ORF1b ORF2 is ORF2
    # PV999257 is actually ORF1a
    exceptions_1a = ["PV999257"]
    #AB518702 AB518703 is actually RdRp
    exceptions_1b = []
    
    # dictionary with ORF coordinates
    dict_coord = {}


    

    with open(input_file) as handle:
        records = list(SeqIO.parse(handle, 'gb'))

        # entries with no annotations
        no_annot = 0
        # entries with no annotations of target CDS
        no_annot_target = 0
        # CDS with no annotations of target ORFs
        notarget_annot_CDS = 0
        # CDS with no annotations
        no_annot_CDS = 0

        # Entries with CDS that has annotation "nonstructural protein"
        entries_onlynonstrprotein = []
        entries_onlynonstrprotein_short = []
        entries_conflicting_annotations = []
        entries_noannot = []
        entries_notarget_annot = []
        
        for rec in records:

            # Omit unverified sequences
            if "UNVERIFIED" in rec.description:
                continue
            # The coordinates of artificial sequences will not be extracted
            artificial_condition1 = ("patent" in rec.description.lower())
            artificial_condition2 = ("crispr" in rec.description.lower()) or ("oligonucleotide" in rec.description.lower())
            artificial_condition3 = "method" in rec.description.lower()
            artificial_condition4 = "modified microbial nucleic acid" in rec.description.lower()
            # Omit artificial sequences
            if artificial_condition1 or artificial_condition2 or artificial_condition3 or artificial_condition4:
                continue

            
            dict_coord[rec.name] = {}
            # iterate over record features

            # number of CDS
            cds_count = 0
            # number of annotated CDS with target ORFs
            cds_annot_count = 0

            # Set coordinates for nonstructural protein which type cannot be inferred from 
            NSP_start, NSP_end, NSP_strand = None, None, None
            for feature in rec.features:

                # check primers to identify the amplified region
                # !!! REVISE removing Mon269_Mon270
                if feature.type == 'source':
                    Mon340_Mon348 = False
                    Mon269_Mon270 = False
                    if feature.qualifiers.get("PCR_primers") != None:
                        #print(feature.qualifiers["PCR_primers"])
                        if "mon340" in feature.qualifiers["PCR_primers"][0].lower() or "mon348" in feature.qualifiers["PCR_primers"][0].lower():
                            Mon340_Mon348 = True
                            Mon269_Mon270 = False
                        elif "mon269" in feature.qualifiers["PCR_primers"][0].lower() or "mon270" in feature.qualifiers["PCR_primers"][0].lower():
                            Mon340_Mon348 = False
                            Mon269_Mon270 = True
        
                if feature.type == 'CDS':
                    cds_count += 1
                    
                    # Check codon start
                    if 'codon_start' in feature.qualifiers.keys():
                        cod_start = int(feature.qualifiers['codon_start'][0]) - 1
                        if cod_start<0:
                            print('Codon start < 0, something is wrong')
                    else:
                        cod_start = 0

                    # check all annotations in gene, product, note qualifiers
                    CDS_raw_annotations = []
                    for el in ('product', 'gene', 'note'):
                        CDS_raw_annotations.extend(feature.qualifiers.get(el, []))
                    
                    if len(CDS_raw_annotations) != 0:
                        # now we will leave annotations of target ORFs
                        CDS_annotations = list(map(lambda x: map_feature(x, orf_dict), CDS_raw_annotations))
                        CDS_annotations = list(filter(lambda x: x in orf_types, CDS_annotations))

                        # TARGET ORF ANNOTATIONS WERE  FOUND
                        if len(CDS_annotations) != 0:
                                                    
                            #print("Annotations were successfully found")
                            #print(CDS_annotations)

                            # check for consistency of CDS annotations
                            uniq_annotations = list(set(CDS_annotations))
                            if len(uniq_annotations) == 1:
                                CDS_product = uniq_annotations[0]
                                # CDS is a target ORF, the coordinates are not joined
                                if CDS_product in orf_types_final:
                                    dict_coord[rec.name][CDS_product] = [int(feature.location.start) + cod_start, int(feature.location.end)]
                                    dict_coord[rec.name][CDS_product + '-strand'] = feature.location.strand
                                    cds_annot_count +=1
                                elif CDS_product == "1AB":

                                    # check whether the coordinates are joined
                                    if len(feature.location.parts) == 2:
                                        dict_coord[rec.name]['1A'] = [int(feature.location.parts[0].start) + cod_start, int(feature.location.parts[0].end)]
                                        dict_coord[rec.name]['1A-strand'] = feature.location.parts[0].strand
                                        dict_coord[rec.name]['1B'] = [int(feature.location.parts[1].start), int(feature.location.parts[1].end)]
                                        dict_coord[rec.name]['1B-strand'] = feature.location.parts[1].strand
                                        cds_annot_count +=1
                                    elif len(feature.location.parts) == 1:
                                        dict_coord[rec.name]['1B'] = [int(feature.location.parts[0].start) + cod_start, int(feature.location.parts[0].end)]
                                        dict_coord[rec.name]['1B-strand'] = feature.location.parts[0].strand
                                        cds_annot_count +=1
                                    else:
                                        print("strange locations in {}".format(rec.name))
                                        print(feature.location)
                            else:
                                print("Conflicting CDS annotations for {}".format(rec.name))
                                #print(CDS_annotations)
                                #print(uniq_annotations)

                                present_1A = ('1A' in uniq_annotations)
                                present_1B = ('1B' in uniq_annotations)
                                present_1A_1B = present_1A and present_1B
                                present_1AB = ('1AB' in uniq_annotations)
                                present_1A_1AB = present_1A and present_1AB
                                present_1B_1AB = present_1B and present_1AB
                                present_2 = ('2' in uniq_annotations)
                                present_1AB_2 = (present_1A and present_2) or (present_1B and present_2) or (present_1AB and present_2) 
                                
                                
                                if present_1AB_2:
                                    if rec.name in exceptions_ORF2:
                                        dict_coord[rec.name]['2'] = [int(feature.location.start) + cod_start, int(feature.location.end)]
                                        dict_coord[rec.name]['2-strand'] = feature.location.strand
                                        cds_annot_count +=1
                                        print("Resolved for ORF2")
                                    else:
                                        print("Could not resolve the conflict")
                                        entries_conflicting_annotations.append(rec.name + ': ORF1ab + ORF2' + ':' + rec.description + '\n')
                                # If ORF1a and ORF1b are present, we choose ORF1b
                                elif present_1A_1B:
                                    if rec.name in exceptions_1a:
                                        dict_coord[rec.name]['1A'] = [int(feature.location.start) + cod_start, int(feature.location.end)]
                                        dict_coord[rec.name]['1A-strand'] = feature.location.strand
                                        cds_annot_count +=1
                                        print("Resolved for ORF1a")
                                    else:
                                        dict_coord[rec.name]['1B'] = [int(feature.location.start) + cod_start, int(feature.location.end)]
                                        dict_coord[rec.name]['1B-strand'] = feature.location.strand
                                        cds_annot_count +=1
                                        #entries_conflicting_annotations.append(rec.name + ': ORF1a + ORF1b' + ':' + rec.description + '\n')
                                        print("Resolved for ORF1b")
                                # ORF1ab with joined coordinates
                                elif len(feature.location.parts) == 2:
                                    dict_coord[rec.name]['1A'] = [int(feature.location.parts[0].start) + cod_start, int(feature.location.parts[0].end)]
                                    dict_coord[rec.name]['1A-strand'] = feature.location.parts[0].strand
                                    dict_coord[rec.name]['1B'] = [int(feature.location.parts[1].start), int(feature.location.parts[1].end)]
                                    dict_coord[rec.name]['1B-strand'] = feature.location.parts[1].strand
                                    cds_annot_count +=1
                                    print("Resolved for ORF1ab")
                                elif present_1A_1AB and ('1A' not in dict_coord[rec.name].keys()):
                                    dict_coord[rec.name]['1A'] = [int(feature.location.start) + cod_start, int(feature.location.end)]
                                    dict_coord[rec.name]['1A-strand'] = feature.location.strand
                                    cds_annot_count +=1
                                    print("Resolved for ORF1a")
                                elif present_1B_1AB and ('1B' not in dict_coord[rec.name].keys()):
                                    dict_coord[rec.name]['1B'] = [int(feature.location.start) + cod_start, int(feature.location.end)]
                                    dict_coord[rec.name]['1B-strand'] = feature.location.strand
                                    cds_annot_count +=1
                                    print("Resolved for ORF1b")
                                elif '1B' in dict_coord[rec.name].keys() or '1A' in dict_coord[rec.name].keys():
                                    print("The coordinates've been extracted from another CDS")
                            if rec.name == "MF973517":
                                print("MF973517")
                                print(cds_annot_count)
                                print(dict_coord[rec.name])
                        # NO TARGET ORF ANNOTATIONS WERE FOUND
                        else:            
                            #print("No annotations of target ORFs of CDS in {}".format(rec.name))
                            #print(CDS_raw_annotations)
                            #print(rec.description)
                            notarget_annot_CDS +=1

                            if "nonstructural protein" in CDS_raw_annotations:
                                #print("CDS of {} has the only annotation \'nonstructural protein\' ".format(rec.name))
                                if len(rec) > 300:
                                    if "orf1a" in rec.description.lower():
                                        dict_coord[rec.name]["1A"] = [int(feature.location.start) + cod_start, int(feature.location.end)]
                                        dict_coord[rec.name]['1A-strand'] = feature.location.strand
                                        cds_annot_count +=1
                                        cds_count +=1
                                    # sequences covers ORF1b and ORF2
                                    elif  len(rec)< 700 and ("capsid" in rec.description.lower()):
                                        dict_coord[rec.name]["1B"] = [int(feature.location.start) + cod_start, int(feature.location.end)]
                                        dict_coord[rec.name]['1B-strand'] = feature.location.strand
                                        cds_annot_count +=1
                                        cds_count +=1
                                    else:
                                        # ORF1b with joined coordinates
                                        #print(feature.location.parts)
                                        if len(feature.location.parts) == 2:
                                            dict_coord[rec.name]['1A'] = [int(feature.location.parts[0].start) + cod_start, int(feature.location.parts[0].end)]
                                            dict_coord[rec.name]['1A-strand'] = feature.location.parts[0].strand
                                            dict_coord[rec.name]['1B'] = [int(feature.location.parts[1].start), int(feature.location.parts[1].end)]
                                            dict_coord[rec.name]['1B-strand'] = feature.location.parts[1].strand
                                            cds_annot_count +=1
                                        else:
                                            NSP_start = int(feature.location.start) + cod_start
                                            NSP_end = int(feature.location.parts[0].end)
                                            NSP_strand = feature.location.strand
                                            #entries_onlynonstrprotein.append(rec.name + ':' + str(len(rec)) + ":"  +rec.description + '\n')
                                            # Тут записать координаты рамки и в конце проверить, нет ли других аннотаций

                                else:
                                    if Mon340_Mon348 == True:
                                        dict_coord[rec.name]["1A"] = [int(feature.location.start) + cod_start, int(feature.location.end)]
                                        dict_coord[rec.name]['1A-strand'] = feature.location.strand
                                        cds_annot_count +=1
                                        cds_count +=1
                                    else:
                                        '''
                                        These sequences cover ORF1a
                                        KX522573
                                        KX522572
                                        KX522571
                                        KX522570
                                        KX522569
                                        KX522568
                                        KX522567
                                        KX522566
                                        KP090043
                                        KP090042
                                        KP090041
                                        KP090040
                                        KP090039
                                        KP090038
                                        KP090037
                                        KP090036
                                        KP090035
                                        '''
                                        if "KX5225" in rec.name or "KP0900" in rec.name:
                                            dict_coord[rec.name]["1A"] = [int(feature.location.start) + cod_start, int(feature.location.end)]
                                            dict_coord[rec.name]['1A-strand'] = feature.location.strand
                                            cds_annot_count +=1
                                            cds_count +=1
                                        else:
                                            entries_onlynonstrprotein_short.append(rec.name + ':' + rec.description + '\n')
                    else:
                        #print("No annotations of CDS in {}".format(rec.name))
                        #print(rec.description)
                        #print(feature)
                        no_annot_CDS +=1

            # Determining the type of ORF indicated as "nonstructural protein"
            if NSP_start != None:
                present_1A = ('1A' in dict_coord[rec.name].keys())
                present_1B = ('1B' in dict_coord[rec.name].keys())
                present_2 = ('2' in dict_coord[rec.name].keys())
                present_1B2 = present_1B and present_2
                present_1A2 = present_1A and present_2

                if present_1B2:
                    dict_coord[rec.name]["1A"] = [NSP_start, NSP_end]
                    dict_coord[rec.name]['1A-strand'] = NSP_strand
                    cds_annot_count +=1
                    cds_count +=1
                elif present_1A2:
                    dict_coord[rec.name]["1B"] = [NSP_start, NSP_end]
                    dict_coord[rec.name]['1B-strand'] = NSP_strand
                    cds_annot_count +=1
                    cds_count +=1
                elif present_1A:
                    dict_coord[rec.name]["1B"] = [NSP_start, NSP_end]
                    dict_coord[rec.name]['1B-strand'] = NSP_strand
                    cds_annot_count +=1
                    cds_count +=1
                elif present_2:
                    dict_coord[rec.name]["1B"] = [NSP_start, NSP_end]
                    dict_coord[rec.name]['1B-strand'] = NSP_strand
                    cds_annot_count +=1
                    cds_count +=1
                elif present_1B:
                    dict_coord[rec.name]["1A"] = [NSP_start, NSP_end]
                    dict_coord[rec.name]['1A-strand'] = NSP_strand
                    cds_annot_count +=1
                    cds_count +=1
                else:
                    entries_onlynonstrprotein.append(rec.name + ':' + str(len(rec)) + ":"  +rec.description + '\n')

            
            '''
            # Check whether seq encodes ORF2
            if Mon269_Mon270 == True:
                print("Mon ORF2 primers: {}".format(rec.name))
                #dict_coord[rec.name]['2'] = [int(feature.location.start) + cod_start, int(feature.location.end)]
                #dict_coord[rec.name]['2-strand'] = feature.location.strand
                #cds_annot_count +=1    
            '''


            if cds_annot_count == 0 or cds_count == 0:
                # Check the keywords in DEFINITION
                if len(rec) < 2700:
                    ORFs = map_keywords(rec.description,description_keywords)
                    if len(ORFs) == 1:
                        ORF_code = ORFs[0]
                        for feature in rec.features:
                            if feature.type == 'misc_feature':
                                dict_coord[rec.name][ORF_code] = [int(feature.location.start) + cod_start, int(feature.location.end)]
                                dict_coord[rec.name][ORF_code + '-strand'] = feature.location.strand
                                cds_annot_count +=1
                                cds_count +=1
                    elif len(ORFs) == 0:
                        print("No keywords were found in description for {}".format(rec.name))
                    else:
                        print("Conflicting keywords were found in description for {}".format(rec.name))
                        print(ORFs)
                else:
                    # Check misc features for annotations
                    for feature in rec.features:
                        if feature.type == "misc_feature":
                            miscfeature_raw_annotations = []
                            for el in ('product', 'gene', 'note'):
                                miscfeature_raw_annotations.extend(feature.qualifiers.get(el, []))
                            #print(miscfeature_raw_annotations)
                            
                            miscfeature_annotations = []
                            for f in miscfeature_raw_annotations:
                                ORFs = map_keywords(f,description_keywords)
                                miscfeature_annotations.extend(ORFs)
                            miscfeature_annotations = list(set(miscfeature_annotations))
                            if len(miscfeature_annotations) != 1:
                                print("Conflicting annotations in misc_feature for {}".format(rec.name))
                                print(miscfeature_raw_annotations)
                                print(miscfeature_annotations)
                            else:
                                ORF_code = miscfeature_annotations[0]
                                dict_coord[rec.name][ORF_code] = [int(feature.location.start) + cod_start, int(feature.location.end)]
                                dict_coord[rec.name][ORF_code + '-strand'] = feature.location.strand
                                cds_annot_count +=1
                                cds_count +=1

            if cds_annot_count == 0:
                no_annot_target +=1
                entries_notarget_annot.append(rec.name + ':' + str(len(rec)) + ':' + rec.description + '\n')
            if cds_count == 0:
                no_annot +=1
                entries_noannot.append(rec.name + ':' + str(len(rec)) + ':' + rec.description + '\n')
           



        # Write extracted coordinates to file
        with open(out_file_name, 'w') as out_file:
            line = "GBAC"
            for ORF in orf_types_final:
                line = line + ',' + ORF + ',' + ORF + '-strand'
            out_file.write(line + '\n')
            for rec_id, values in dict_coord.items():
                s = rec_id
                for orf in orf_types_final:
                    if orf in values:
                        s += f",{values[orf][0]}-{values[orf][1]},{values[orf+'-strand']}"
                    else:
                        s += ",NA-NA,NA"
                s += "\n"
                out_file.write(s)
        out_file.close()


        print("Entries with no CDS annotation: {}".format(no_annot))
        print("Entries with no CDS with target ORF annotation: {}".format(no_annot_target))
        print("CDS with no target ORFs: {}".format(notarget_annot_CDS))
        print("CDS with no annotations: {}".format(no_annot_CDS))

        with open(out_file_name_temp + '_noannotCDS.txt', 'w') as file:
            file.writelines(entries_noannot)

        with open(out_file_name_temp + '_noannot_targetCDS.txt', 'w') as file:
            file.writelines(entries_notarget_annot)
            
        with open(out_file_name_temp + '_noannot_targetCDS.txt', 'w') as file:
            file.writelines(entries_notarget_annot)

        with open(out_file_name_temp + '_conflictingannot.txt', 'w') as file:
            file.writelines(entries_conflicting_annotations)

        with open(out_file_name_temp + '_onlyNSP_long.txt', 'w') as file:
            file.writelines(entries_onlynonstrprotein)
            
        with open(out_file_name_temp + '_onlyNSP_short.txt', 'w') as file:
            file.writelines(entries_onlynonstrprotein_short)

        
        return 0


In [9]:
file_gb = fetch_seq_from_Nucleotide('"txid39733"[Organism]' , "data/Astroviridae_15102025.gb")

Query to GenBank Nucleotide database:""txid39733"[Organism]"
Number of records found: 16012
16012 records will be downloaded
Downloaded 500 sequences
Downloaded 1000 sequences
Downloaded 1500 sequences
Downloaded 2000 sequences
Downloaded 2500 sequences
Downloaded 3000 sequences
Downloaded 3500 sequences
Downloaded 4000 sequences
Downloaded 4500 sequences
Downloaded 5000 sequences
Downloaded 5500 sequences
Downloaded 6000 sequences
Downloaded 6500 sequences
Downloaded 7000 sequences
Downloaded 7500 sequences
Downloaded 8000 sequences
Downloaded 8500 sequences
Downloaded 9000 sequences
Downloaded 9500 sequences
Downloaded 10000 sequences
Downloaded 10500 sequences
Downloaded 11000 sequences
Downloaded 11500 sequences
Downloaded 12000 sequences
Downloaded 12500 sequences
Downloaded 13000 sequences
Downloaded 13500 sequences
Downloaded 14000 sequences
Downloaded 14500 sequences
Downloaded 15000 sequences
Downloaded 15500 sequences
Downloaded 16000 sequences
Finished


In [118]:
input_file = "data/Astroviridae_15102025.gb"
data = fetch_metadata_from_gb(input_file)

In [10]:
data[data["Artificial"] == "Yes"]

,Version,GenBank title,Release date,Organism name,Virus Lineage,Species,Family,Length,Submitters,Strain,Isolate,Geo location,Country,Collection date,Tissue/Specimen/Source,Host,Environmental,CDS count,Artificial,UNVERIFIED
PG036827,PG036827.1,JP 2022536474-A/8: Methods and Compositions of...,28-FEB-2025,Human astrovirus,Viruses;Riboviria;Orthornavirae;Pisuviricota;S...,NA,Astroviridae,5388,NaN,NA,NA,NA,NA,NA,NA,NA,No,0,Yes,No
PG036826,PG036826.1,JP 2022536474-A/7: Methods and Compositions of...,28-FEB-2025,Human astrovirus,Viruses;Riboviria;Orthornavirae;Pisuviricota;S...,NA,Astroviridae,5412,NaN,NA,NA,NA,NA,NA,NA,NA,No,0,Yes,No
PG036825,PG036825.1,JP 2022536474-A/6: Methods and Compositions of...,28-FEB-2025,Human astrovirus,Viruses;Riboviria;Orthornavirae;Pisuviricota;S...,NA,Astroviridae,5358,NaN,NA,NA,NA,NA,NA,NA,NA,No,0,Yes,No
PG036824,PG036824.1,JP 2022536474-A/5: Methods and Compositions of...,28-FEB-2025,Human astrovirus,Viruses;Riboviria;Orthornavirae;Pisuviricota;S...,NA,Astroviridae,5358,NaN,NA,NA,NA,NA,NA,NA,NA,No,0,Yes,No
PG036823,PG036823.1,JP 2022536474-A/4: Methods and Compositions of...,28-FEB-2025,Human astrovirus,Viruses;Riboviria;Orthornavirae;Pisuviricota;S...,NA,Astroviridae,5337,NaN,NA,NA,NA,NA,NA,NA,NA,No,0,Yes,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GN354831,GN354831.1,Sequence 4595 from Patent WO2007130519,12-MAY-2009,Mamastrovirus 13,Viruses;Riboviria;Orthornavirae;Pisuviricota;S...,Mamastrovirus ovis,Astroviridae,60,NaN,NA,NA,NA,NA,NA,NA,NA,No,0,Yes,No
GN354830,GN354830.1,Sequence 4594 from Patent WO2007130519,12-MAY-2009,Mamastrovirus 13,Viruses;Riboviria;Orthornavirae;Pisuviricota;S...,Mamastrovirus ovis,Astroviridae,60,NaN,NA,NA,NA,NA,NA,NA,NA,No,0,Yes,No
GN354829,GN354829.1,Sequence 4593 from Patent WO2007130519,12-MAY-2009,Mamastrovirus 13,Viruses;Riboviria;Orthornavirae;Pisuviricota;S...,Mamastrovirus ovis,Astroviridae,60,NaN,NA,NA,NA,NA,NA,NA,NA,No,0,Yes,No
DL470173,DL470173.1,ANTISENSE ANTIVIRAL COMPOUND AND METHOD FOR TR...,22-DEC-2008,Human astrovirus,Viruses;Riboviria;Orthornavirae;Pisuviricota;S...,NA,Astroviridae,40,NaN,NA,NA,NA,NA,NA,NA,NA,No,0,Yes,No


In [113]:
orf_coord("data/Astroviridae_15102025.gb", "ORF_names.csv")

D:\Virology\RNA_viruses\Astrovirus\Astroviridae_database\Astroviridae_15102025_orf-coords.csv
No keywords were found in description for PX395426
No keywords were found in description for PX395425
No keywords were found in description for PX395424
No keywords were found in description for PX395423
No keywords were found in description for PX395422
No keywords were found in description for OR951076
No keywords were found in description for OR951065
No keywords were found in description for OR951063
No keywords were found in description for OR951056
No keywords were found in description for OR951044
No keywords were found in description for OR951043
No keywords were found in description for OR951029
No keywords were found in description for OR951024
No keywords were found in description for OR951013
No keywords were found in description for PX095754
No keywords were found in description for PX113254
Conflicting CDS annotations for PV999257
Resolved for ORF1b
Conflicting annotations in mis

0

Два раза встречается ORF1ab polyprotein в PX095750

In [29]:
def split_genome_and_reverse(input_file, coord_file):
    '''
    Splits nucleotide sequences from input_file into ORFs using their coordinates from 
    coord_file

    Input:
        input_file - file with nucleotide sequences in fasta-format
        coord_file - text file with coordinates of NTRs

    '''
    

    coord_df = pd.read_csv(coord_file, sep=",", index_col=0)
    #orfs = list(coord_df.columns)
    orfs = ['1A', '1B', '2']
    print(coord_df)
    dict_orfs = {}
    for orf in orfs:
        dict_orfs[orf] = list()
    seqs = SeqIO.parse(open(input_file), format='fasta')
    for seq in seqs:
        # accession number of sequence
        acc = seq.id.split("_")[0]
        if acc == 'NC' or acc == 'AC':
            acc = '_'.join([acc,seq.id.split("_")[1]])
        for orf in orfs:            
            if (acc in coord_df.index) and (coord_df.loc[acc,orf] != 'NA-NA'):
                cd = coord_df[orf][acc]
                s, e = [int(x) for x in cd.split('-')]
                seq_orf = copy.deepcopy(seq)
                seq_orf.seq = seq.seq[s:e]
                strand_col = f"{orf}-strand"
                if strand_col in coord_df.columns:
                    strand_value = coord_df.loc[acc, strand_col]
                    if strand_value == -1:
                        seq_orf.seq = seq_orf.seq.reverse_complement()
                        seq_orf.id += "_reverse"
                        seq_orf.description = seq_orf.id
                        print(acc)
                dict_orfs[orf].append(seq_orf)
            else:
                print('{} not in coord file'.format(acc))
    out_temp = os.path.splitext(input_file)[0]
    for orf in dict_orfs.keys():
        outfile = out_temp + '_' + orf + '.fasta'
        SeqIO.write(dict_orfs[orf], outfile, "fasta")

In [30]:
input_file = "data/Astroviridae_15102025.fasta"
coord_file = "data/Astroviridae_15102025_orf-coords.csv"

In [31]:
split_genome_and_reverse(input_file,coord_file)

               1A  1A-strand         1B  1B-strand          2  2-strand
GBAC                                                                   
PV418556   0-2670        1.0  2621-4157        1.0  3957-6444       1.0
PV418555   0-2667        1.0  2618-4154        1.0  3954-6466       1.0
PV418548    NA-NA        NaN      NA-NA        NaN   116-2447       1.0
PV418547    NA-NA        NaN      NA-NA        NaN    57-2416       1.0
PV418557  59-2606        1.0  2569-4138        1.0  4091-6455       1.0
...           ...        ...        ...        ...        ...       ...
Y08628      NA-NA        NaN      NA-NA        NaN      1-220       1.0
Y08627      NA-NA        NaN      NA-NA        NaN      1-220       1.0
Z66541      NA-NA        NaN      NA-NA        NaN     0-2349       1.0
Z46658      NA-NA        NaN      NA-NA        NaN     0-2337       1.0
HUANSSPS    NA-NA        NaN      NA-NA        NaN      NA-NA       NaN

[15516 rows x 6 columns]
PV418548 not in coord file
PV418548 no